# Face Regocnition Project
## Introduction
The project is about face recognition. Limited by the computational power of Jetson Nano, our group choose this ultra light-weight model to recognize faces. Thus, the frame rate for the model reaches a relatively high level. However, the model is not very accurate. One mean drawback of this model is that it is not able to differentiate between different people and virtual characters. For example, the pretrained model provided by the author recognizes milk dragon as a person. It also behaves poor when recognizing other comic characters.

The picture shows an example improvement of our work.

<img src="./examples/slim-320-1.png" alt="Facial detection" width="900" />
<img src="./examples/slim-320-2.png" alt="Facial detection" width="900" />

Our project deal with this problem by training the model with labeled classes on our own dataset.

## Inference with trained model

In [ ]:
import os
import sys
import tqdm

from models.ssd.config.fd_config import define_img_size

### Configurations for model and data

In [ ]:
class Config:
    def __init__(self):
        self.net_type = "RFB"  # The network architecture, optional: RFB or slim
        self.input_size = 320  # Network input size, e.g., 128/160/320/480/640/1280
        self.threshold = 0.6  # Score threshold
        self.candidate_size = 1500  # NMS candidate size
        self.on_board = False  # Run on board
        self.width = 640  # Width of camera
        self.height = 480  # Height of camera
        self.model_path = f"./checkpoints/version-rfb-{self.input_size}.pth"  # Path to the trained model
        self.label_path = "./checkpoints/version-labels.txt"  # Path to the labels

    def __str__(self):
        config_str = "\n".join([f"{key}: {value}" for key, value in self.__dict__.items()])
        return f"Config:\n{config_str}"
config  = Config()

define_img_size(config.input_size)
from models.ssd.mb_tiny_fd import create_mb_tiny_fd, create_mb_tiny_fd_predictor
from models.ssd.mb_tiny_RFB_fd import create_Mb_Tiny_RFB_fd, create_Mb_Tiny_RFB_fd_predictor

### Load model

In [ ]:
# load model
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
define_img_size(config.input_size)
class_names = [name.strip() for name in open(config.label_path).readlines()]

if config.net_type == 'slim':
    model_path = config.model_path
    net = create_mb_tiny_fd(len(class_names), is_test=True, device=device)
    predictor = create_mb_tiny_fd_predictor(net, candidate_size=config.candidate_size, device=device)
elif config.net_type == 'RFB':
    model_path = config.model_path
    net = create_Mb_Tiny_RFB_fd(len(class_names), is_test=True, device=device)
    predictor = create_Mb_Tiny_RFB_fd_predictor(net, candidate_size=config.candidate_size, device=device)
else:
    print("The net type is wrong!")
    sys.exit(1)
net.load(model_path)


### Begin inference
You should prepare the image folder and the save dirctory.

In [ ]:
from infer import detect

folder = "./examples/imgs"
save_dir = f"./examples/untrained-rfb-{config.input_size}"
os.makedirs(save_dir, exist_ok=True)
imgs = os.listdir(folder)
for img in tqdm.tqdm(imgs, total=len(imgs)):
    detect(os.path.join(folder, img), os.path.join(save_dir, img), predictor, class_names, config.candidate_size, config.threshold)

## Train the model with customed dataset
### Dataset format
You should not worry about the format of the dataset, just make sure that you have a folder named "split" and a file named "labels.txt". The "split" folder should contain 3 folders named "train.txt", "val.txt", "test.txt", telling the model images use for training, validation and testing. A detailed annotation for the ground truth box and class is required. The "labels.txt" file should contain the class name for the classes you have labeled in the dataset.

An example for spliting and label can be found in the folder "example".

### Model training
You can either ues the `train.bash` file to directly train the model or use the following code to train the model.


In [ ]:
import logging
import sys

from train import main
from models.ssd.config.fd_config import define_img_size

import torch

In [ ]:
class TrainConfig:
    def __init__(self):
        # General Settings
        self.dataset_type = "diy"  # Specify dataset type. Currently support voc.
        self.data_split = "cartoon/split"  # Dataset directory path
        self.data_base = "DATA_DIR"  # Dataset directory path
        self.balance_data = False  # Balance training data by down-sampling more frequent labels.

        # Network Settings
        self.net = "RFB"  # The network architecture, optional (RFB, slim)
        self.freeze_base_net = False  # Freeze base net layers.
        self.freeze_net = False  # Freeze all the layers except the prediction head.
        self.cuda_index = None  # CUDA index for multiple GPU training

        # Optimizer Parameters (SGD)
        self.lr = 1e-10  # Initial learning rate
        self.momentum = 0.9  # Momentum value for optimizer
        self.weight_decay = 5e-4  # Weight decay for SGD
        self.gamma = 0.1  # Gamma update for SGD
        self.base_net_lr = None  # Initial learning rate for base net.
        self.extra_layers_lr = None  # Initial learning rate for the layers not in base net and prediction heads.

        # Pretrained Models and Checkpoints
        self.base_net = None  # Pretrained base model
        self.pretrained_ssd = None  # Pre-trained SSD model
        self.resume = None  # Checkpoint state_dict file to resume training from

        # Scheduler Settings
        self.scheduler = "multi-step"  # Scheduler for SGD. Options: 'multi-step', 'cosine'
        self.milestones = "95,150"  # Milestones for MultiStepLR
        self.t_max = 120.0  # T_max value for Cosine Annealing Scheduler.

        # Training Parameters
        self.batch_size = 256  # Batch size for training
        self.num_epochs = 160  # Number of epochs
        self.num_workers = 16  # Number of workers used in dataloading
        self.validation_epochs = 5  # Number of epochs between validations

        # Logging and Checkpoints
        self.checkpoint_folder = 'my_trained/'  # Directory for saving checkpoint models
        self.log_dir = './logs'  # Log directory

        # Additional Parameters
        self.power = 2  # Poly learning rate power
        self.overlap_threshold = 0.35  # Overlap threshold
        self.optimizer_type = "SGD"  # Optimizer type
        self.input_size = 320  # Define network input size (options: 128, 160, 320, 480, 640, 1280)
trainConfig = TrainConfig()


In [ ]:

define_img_size(trainConfig.input_size)
from models.ssd.mb_tiny_fd import create_mb_tiny_fd, create_mb_tiny_fd_predictor
from models.ssd.mb_tiny_RFB_fd import create_Mb_Tiny_RFB_fd, create_Mb_Tiny_RFB_fd_predictor

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
input_img_size = trainConfig.input_size  # define input size ,default optional(128/160/320/480/640/1280)
logging.info("inpu size :{}".format(input_img_size))


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
logging.info(f"Use {DEVICE}.")
if DEVICE == "cuda":
    torch.backends.cudnn.benchmark = True

main(trainConfig, create_mb_tiny_fd, create_Mb_Tiny_RFB_fd, DEVICE)

## Real-time detectoin using model with camera video stream
Run `python run.py` to start real-time detection. You can set the parameters specify the model path and so on. Feel free to check the code and find more details yourselves!